In [1]:
import numpy as np
import pandas as pd

from collections import defaultdict
from collections import Counter

import re

In [2]:
data_dir = "../data/Liu_2019"

hla_ii_file = "HLA_II_03042023.csv"
ID_file = "SraRunTable_short.tsv"
table_s1 = "41591_2019_654_MOESM4_ESM.xlsx"

In [5]:
df_hla = pd.read_csv(data_dir+"/"+hla_ii_file, header=0)
print(df_hla.shape)
df_hla[:6]

(287, 11)


,ID,DRB_1,DRB_2,DQA_1,DQA_2,DQB_1,DQB_2,DPA_1,DPA_2,DPB_1,DPB_2
0,SRR10801640,DRB1*15:01,DRB1*03:01,DQA1*01:02,DQA1*05:01,DQB1*06:02,DQB1*02:01,DPA1*01:03,DPA1*02:01,DPB1*04:01,DPB1*01:01
1,SRR10801641,DRB1*15:02,DRB1*14:01,DQA1*01:04,DQA1*01:03,DQB1*06:01,DQB1*05:03,DPA1*01:03,DPA1*02:01,DPB1*13:01,DPB1*04:01
2,SRR10801642,DRB1*15:01,DRB1*13:01,DQA1*01:02,DQA1*01:03,DQB1*06:02,DQB1*06:03,DPA1*01:03,NaN,DPB1*02:01,DPB1*03:01
3,SRR10801643,DRB1*11:04,DRB1*04:04,DQA1*03:01,DQA1*05:05,DQB1*03:02,DQB1*03:01,DPA1*01:03,NaN,DPB1*04:01,NaN
4,SRR10801644,DRB1*13:03,DRB1*03:01,DQA1*05:01,DQA1*05:05,DQB1*03:01,DQB1*02:01,DPA1*01:03,NaN,DPB1*104:01,NaN
5,SRR10801645,DRB1*11:04,DRB1*11:01,DQA1*05:05,NaN,DQB1*03:01,NaN,DPA1*01:03,NaN,DPB1*04:01,NaN


In [6]:
df_ID = pd.read_csv(data_dir+"/"+"SraRunTable_short.tsv",sep='\t')
print(df_ID.shape)
df_ID[:6]

(288, 41)


,Run,analyte_type,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,biospecimen_repository,biospecimen_repository_sample_id,Bytes,...,submitted_subject_id,AssemblyName,study_disease,subject_is_affected,flowcell_barcode (run),instrument_name (run),lsid (exp),lsid (run),material_type (exp),biospecimen__repository_sample_type
0,SRR11080170,DNA; RNA,WXS,152,21110407976,PRJNA82747,SAMN12309122,NHGRI_Melanoma_Garraway,Patient142_T,6369252802,...,Patient142,GCA_000001405.13,Melanoma,Yes,"HK5C7BBXX\, HJYHVBBXX",SL-HFC,broadinstitute.org:bsp.prod.sample:EXZ57,broadinstitute.org:bsp.prod.sample:EXZ57,DNA:DNA Somatic,T
1,SRR11080191,DNA; RNA,WXS,152,24884983848,PRJNA82747,SAMN12309302,NHGRI_Melanoma_Garraway,Patient156_T,7505282208,...,Patient156,GCA_000001405.13,Melanoma,Yes,"HJYHVBBXX\, HK5C7BBXX",SL-HFC,broadinstitute.org:bsp.prod.sample:EXZ6D,broadinstitute.org:bsp.prod.sample:EXZ6D,DNA:DNA Somatic,T
2,SRR11080225,DNA; RNA,WXS,152,38197538744,PRJNA82747,SAMN12309266,NHGRI_Melanoma_Garraway,Patient21_T_M,7901881806,...,Patient21,GCA_000001405.13,Melanoma,Yes,"HHKHGBBXX\, HHKHKBBXX\, HHK23BBXX\, HHK22BBXX","SL-HFB\, SL-HFA",broadinstitute.org:bsp.prod.sample:DPR96,broadinstitute.org:bsp.prod.sample:DPR96,DNA:DNA Somatic,T_M
3,SRR11080226,DNA; RNA,WXS,152,26446046040,PRJNA82747,SAMN12309293,NHGRI_Melanoma_Garraway,Patient149_T,6976852028,...,Patient149,GCA_000001405.13,Melanoma,Yes,"HKKTJBBXX\, HKHVFBBXX\, HKKLMBBXX","SL-HFC\, SL-HFA",broadinstitute.org:bsp.prod.sample:F56LN,broadinstitute.org:bsp.prod.sample:F56LN,DNA:DNA Somatic,T
4,SRR10841991,DNA,WXS,152,32636008616,PRJNA82747,SAMN12309077,NHGRI_Melanoma_Garraway,Patient100_N,7086590983,...,Patient100,GCA_000001405.13,Melanoma,Yes,"HHKH7BBXX\, HHK22BBXX\, HHKHHBBXX","SL-HFA\, SL-HFB",broadinstitute.org:bsp.prod.sample:DPR7K,broadinstitute.org:bsp.prod.sample:DPR7K,DNA:DNA Somatic,N
5,SRR10841995,DNA,WXS,152,25095568448,PRJNA82747,SAMN12309100,NHGRI_Melanoma_Garraway,Patient112_N,5053428698,...,Patient112,GCA_000001405.13,Melanoma,Yes,"HHK23BBXX\, HHK22BBXX\, HHKHKBBXX\, HHKHGBBXX","SL-HFB\, SL-HFA",broadinstitute.org:bsp.prod.sample:DPSS4,broadinstitute.org:bsp.prod.sample:DPSS4,DNA:DNA Somatic,N


In [7]:
df_ID.columns

Index(['Run', 'analyte_type', 'Assay Type', 'AvgSpotLen', 'Bases',
       'BioProject', 'BioSample', 'biospecimen_repository',
       'biospecimen_repository_sample_id', 'Bytes', 'Center Name',
       'Consent_Code', 'Consent', 'DATASTORE filetype', 'DATASTORE provider',
       'DATASTORE region', 'Experiment', 'gap_accession', 'Instrument',
       'Is_Tumor', 'Library Name', 'LibraryLayout', 'LibrarySelection',
       'LibrarySource', 'Organism', 'Platform', 'ReleaseDate', 'Sample Name',
       'SRA Study', 'study_design', 'study_name', 'submitted_subject_id',
       'AssemblyName', 'study_disease', 'subject_is_affected',
       'flowcell_barcode (run)', 'instrument_name (run)', 'lsid (exp)',
       'lsid (run)', 'material_type (exp)',
       'biospecimen__repository_sample_type'],
      dtype='object')

In [8]:
Counter(df_ID['Is_Tumor'])

Counter({'Yes': 144, 'No': 144})

In [9]:
set(df_hla.ID) - set(df_ID.Run)

set()

In [10]:
set(df_ID.Run) - set(df_hla.ID)

{'SRR10899714'}

In [11]:
set(["_".join(x.split("_")[1:]) for x in df_ID.biospecimen_repository_sample_id.tolist()])

{'N', 'T', 'T_M', 'T_P'}

In [12]:
Counter(["_".join(x.split("_")[1:]) for x, y in zip(df_ID.biospecimen_repository_sample_id.tolist(), 
                                                    df_ID.Is_Tumor.tolist()) if y=="Yes"])

Counter({'T': 56, 'T_M': 49, 'T_P': 39})

In [13]:
Counter(["_".join(x.split("_")[1:]) for x, y in zip(df_ID.biospecimen_repository_sample_id.tolist(), 
                                                    df_ID.Is_Tumor.tolist()) if y=="No"])

Counter({'N': 144})

In [14]:
set(Counter(df_ID.submitted_subject_id.tolist()).values())

{2}

In [15]:
# build a dictionary with submitted subject id (patient***) as key and 
# biospecimen_repository_sample_id (patient***_ one of {'N', 'T', 'T_M', 'T_P'}) as value
# build another dictioary with biospecimen_repository_sample_id and 
# the corresponding run (SRR********) as value

bio_dict = defaultdict(list)

for b, c in zip(df_ID.biospecimen_repository_sample_id.tolist(), 
                df_ID.submitted_subject_id.tolist()):
    bio_dict[c] += [b]


run_dict = defaultdict(str)

for a, b in zip(df_ID.Run.tolist(), 
                df_ID.biospecimen_repository_sample_id.tolist()):
    run_dict[b] = a

In [16]:
bio_dict

defaultdict(list,
            {'Patient142': ['Patient142_T', 'Patient142_N'],
             'Patient156': ['Patient156_T', 'Patient156_N'],
             'Patient21': ['Patient21_T_M', 'Patient21_N'],
             'Patient149': ['Patient149_T', 'Patient149_N'],
             'Patient100': ['Patient100_N', 'Patient100_T_P'],
             'Patient112': ['Patient112_N', 'Patient112_T_P'],
             'Patient102': ['Patient102_N', 'Patient102_T_P'],
             'Patient108': ['Patient108_N', 'Patient108_T_P'],
             'Patient104': ['Patient104_N', 'Patient104_T_P'],
             'Patient105': ['Patient105_N', 'Patient105_T_P'],
             'Patient106': ['Patient106_N', 'Patient106_T_P'],
             'Patient10': ['Patient10_N', 'Patient10_T_M'],
             'Patient110': ['Patient110_N', 'Patient110_T_M'],
             'Patient107': ['Patient107_N', 'Patient107_T_P'],
             'Patient59': ['Patient59_N', 'Patient59_T_M'],
             'Patient36': ['Patient36_N', 'Patient36

In [17]:
df_hla.columns.tolist()

['ID',
 'DRB_1',
 'DRB_2',
 'DQA_1',
 'DQA_2',
 'DQB_1',
 'DQB_2',
 'DPA_1',
 'DPA_2',
 'DPB_1',
 'DPB_2']

In [18]:
# build a dict with run (SRR********) as key and the list of HLAs as value

hla_dict = defaultdict(list)

for cur_id, drb1, drb2, dqa1, dqa2, dqb1, dqb2, dpa1, dpa2, dpb1, dpb2 in zip(df_hla.ID.tolist(), 
                                                                              df_hla.DRB_1.tolist(), 
                                                                              df_hla.DRB_2.tolist(), 
                                                                              df_hla.DQA_1.tolist(), 
                                                                              df_hla.DQA_2.tolist(),
                                                                              df_hla.DQB_1.tolist(), 
                                                                              df_hla.DQB_2.tolist(), 
                                                                              df_hla.DPA_1.tolist(), 
                                                                              df_hla.DPA_2.tolist(),
                                                                              df_hla.DPB_1.tolist(), 
                                                                              df_hla.DPB_2.tolist()):
    if drb2 == drb2:
        drbs = [drb1, drb2]
    else:
        drbs = [drb1 for _ in range(2)]
        
    if dqa2 == dqa2:
        dqas = [dqa1, dqa2]
    else:
        dqas = [dqa1 for _ in range(2)]
    
    if dqb2 == dqb2:
        dqbs = [dqb1, dqb2]
    else:
        dqbs = [dqb1 for _ in range(2)]
        
    if dpa2 == dpa2:
        dpas = [dpa1, dpa2]
    else:
        dpas = [dpa1 for _ in range(2)]
    
    if dpb2 == dpb2:
        dpbs = [dpb1, dpb2]
    else:
        dpbs = [dpb1 for _ in range(2)]
        
    drbs.sort()
    dqas.sort()
    dqbs.sort()
    dpas.sort()
    dpbs.sort()
    
    cur_hla_list = drbs+dqas+dqbs+dpas+dpbs
    hla_dict[cur_id] = cur_hla_list

In [19]:
len(hla_dict)

287

In [20]:
print(len(set(df_ID.Run)))
print(len(set(df_ID.biospecimen_repository_sample_id)))

288
288


**Print out the information for Run SRR10899714, which is not finished as of 18:04 on 03052023**

In [21]:
[(x,y,z) for x,y,z,a in zip(df_ID.biospecimen_repository_sample_id.tolist(), 
                            df_ID.submitted_subject_id.tolist(), 
                            df_ID.Is_Tumor.tolist(), 
                            df_ID.Run.tolist()) if a=="SRR10899714"]

[('Patient131_T', 'Patient131', 'Yes')]

In [22]:
# for each patient, check whether the HLAs are consistent between two samples

patient_ids = set(df_ID.submitted_subject_id)
patient_ids = list(patient_ids)
patient_ids.sort()

match_flags = []

for patient in patient_ids:
    
    bio_list = bio_dict[patient]
    bio_list.sort()
    
    run_list_0 = run_dict[bio_list[0]]
    run_list_1 = run_dict[bio_list[1]]
    
    if "SRR10899714" in [run_list_0, run_list_1]:
        if bio_list[0] == 'Patient131_T':
            only_bio = bio_list[1]
        elif bio_list[1] == 'Patient131_T':
            only_bio = bio_list[0]
        else:
            print("Error in finding bio for Patient131_T and SRR10899714")
        
        only_run = run_dict[only_bio]
        if only_run == "SRR10899714":
            print("Error in bio and run mismatch")
            
        print("             ")
        print(patient)
        print("only has one run output")
        print(only_bio+": "+" ".join(hla_dict[only_run]))
    
    else:
        hlas_0 = hla_dict[run_list_0]
        hlas_1 = hla_dict[run_list_1]

        if hlas_0 == hlas_1:
            match_flags += [True]
        else:
            match_flags += [False]
            print("              ")
            print(patient)
            print(bio_list[0]+": "+" ".join(hlas_0))
            print(bio_list[1]+": "+" ".join(hlas_1))

             
Patient131
only has one run output
Patient131_N: DRB1*15:01 DRB1*15:02 DQA1*01:02 DQA1*01:03 DQB1*06:01 DQB1*06:02 DPA1*01:03 DPA1*02:01 DPB1*03:01 DPB1*13:01
              
Patient143
Patient143_N: DRB1*01:01 DRB1*01:01 DQA1*01:01 DQA1*01:01 DQB1*05:01 DQB1*05:01 DPA1*01:03 DPA1*01:03 DPB1*04:02 DPB1*104:01
Patient143_T: DRB1*01:01 DRB1*01:01 DQA1*01:01 DQA1*01:01 DQB1*05:01 DQB1*05:01 DPA1*01:03 DPA1*01:03 DPB1*03:01 DPB1*1037:01
              
Patient148
Patient148_N: DRB1*07:01 DRB1*16:01 DQA1*01:02 DQA1*02:01 DQB1*03:03 DQB1*05:02 DPA1*01:03 DPA1*01:03 DPB1*03:01 DPB1*04:01
Patient148_T: DRB1*07:01 DRB1*16:01 DQA1*01:02 DQA1*02:01 DQB1*03:03 DQB1*05:02 DPA1*01:03 DPA1*01:03 DPB1*03:01 DPB1*1231:01
              
Patient179
Patient179_N: DRB1*03:01 DRB1*04:01 DQA1*03:01 DQA1*05:01 DQB1*02:01 DQB1*03:02 DPA1*01:03 DPA1*01:03 DPB1*04:01 DPB1*04:01
Patient179_T: DRB1*03:01 DRB1*04:01 DQA1*05:01 DQA1*05:01 DQB1*02:01 DQB1*03:457 DPA1*01:03 DPA1*01:03 DPB1*04:01 DPB1*04:01

**initial attempt on deciding which sample to keep for these patients is listed in file** 

    ../Liu_2019/HLA_II_inconsistency.txt

In [23]:
Counter(match_flags)

Counter({True: 134, False: 9})

In [24]:
print(run_dict[bio_dict['Patient143'][0]])
print(run_dict[bio_dict['Patient143'][1]])

SRR10899840
SRR10899817


In [25]:
print(run_dict[bio_dict['Patient148'][0]])
print(run_dict[bio_dict['Patient148'][1]])

SRR10899825
SRR10899815


In [26]:
df_hla

,ID,DRB_1,DRB_2,DQA_1,DQA_2,DQB_1,DQB_2,DPA_1,DPA_2,DPB_1,DPB_2
0,SRR10801640,DRB1*15:01,DRB1*03:01,DQA1*01:02,DQA1*05:01,DQB1*06:02,DQB1*02:01,DPA1*01:03,DPA1*02:01,DPB1*04:01,DPB1*01:01
1,SRR10801641,DRB1*15:02,DRB1*14:01,DQA1*01:04,DQA1*01:03,DQB1*06:01,DQB1*05:03,DPA1*01:03,DPA1*02:01,DPB1*13:01,DPB1*04:01
2,SRR10801642,DRB1*15:01,DRB1*13:01,DQA1*01:02,DQA1*01:03,DQB1*06:02,DQB1*06:03,DPA1*01:03,NaN,DPB1*02:01,DPB1*03:01
3,SRR10801643,DRB1*11:04,DRB1*04:04,DQA1*03:01,DQA1*05:05,DQB1*03:02,DQB1*03:01,DPA1*01:03,NaN,DPB1*04:01,NaN
4,SRR10801644,DRB1*13:03,DRB1*03:01,DQA1*05:01,DQA1*05:05,DQB1*03:01,DQB1*02:01,DPA1*01:03,NaN,DPB1*104:01,NaN
...,...,...,...,...,...,...,...,...,...,...,...
282,SRR10899972,DRB1*01:01,DRB1*07:01,DQA1*01:01,DQA1*02:01,DQB1*05:01,DQB1*02:02,DPA1*01:03,DPA1*02:01,DPB1*04:01,DPB1*01:01
283,SRR11080170,DRB1*11:01,DRB1*07:01,DQA1*02:01,DQA1*05:05,DQB1*03:01,DQB1*02:02,DPA1*01:03,DPA1*02:01,DPB1*04:01,DPB1*17:01
284,SRR11080191,DRB1*13:01,DRB1*11:01,DQA1*01:03,DQA1*05:05,DQB1*06:03,DQB1*03:01,DPA1*01:03,DPA1*02:07,DPB1*19:01,DPB1*04:01
285,SRR11080225,DRB1*13:01,DRB1*01:01,DQA1*01:03,DQA1*01:01,DQB1*06:03,DQB1*05:01,DPA1*01:03,DPA1*02:01,DPB1*04:02,DPB1*17:01


In [27]:
# build a dict with run (SRR********) as key and the string of sorted HLAs as value
# based on hla_dict, this one concatenates the items in the list into a string


hla_str_dict = defaultdict(str)

for cur_id in df_hla.ID.tolist(): 
    
    cur_hla_list = hla_dict[cur_id]
    hla_str_dict[cur_id] = ",".join(cur_hla_list)

In [28]:
len(hla_str_dict)

287

In [29]:
hla_dict['SRR10801640']

['DRB1*03:01',
 'DRB1*15:01',
 'DQA1*01:02',
 'DQA1*05:01',
 'DQB1*02:01',
 'DQB1*06:02',
 'DPA1*01:03',
 'DPA1*02:01',
 'DPB1*01:01',
 'DPB1*04:01']

In [30]:
hla_str_dict['SRR10801640']

'DRB1*03:01,DRB1*15:01,DQA1*01:02,DQA1*05:01,DQB1*02:01,DQB1*06:02,DPA1*01:03,DPA1*02:01,DPB1*01:01,DPB1*04:01'

In [31]:
hla_dict['SRR10899972']

['DRB1*01:01',
 'DRB1*07:01',
 'DQA1*01:01',
 'DQA1*02:01',
 'DQB1*02:02',
 'DQB1*05:01',
 'DPA1*01:03',
 'DPA1*02:01',
 'DPB1*01:01',
 'DPB1*04:01']

In [32]:
hla_str_dict['SRR10899972']

'DRB1*01:01,DRB1*07:01,DQA1*01:01,DQA1*02:01,DQB1*02:02,DQB1*05:01,DPA1*01:03,DPA1*02:01,DPB1*01:01,DPB1*04:01'

In [33]:
# build a dictionary with patient ID as key, and the string of HLAs as key

# for the one patient with only normal sample result out and nine patients with 
# inconsistency between normal sample and tumor sample, follow the intitial 
# attempt to decide which sample to include as listed in file
# HLA_II_inconsistency.txt

final_hla_dict = defaultdict(str)


patient_ids = set(df_ID.submitted_subject_id)
patient_ids = list(patient_ids)
patient_ids.sort()

manual_dict = defaultdict(str)

manual_dict['Patient131'] = 'Patient131_N'
manual_dict['Patient143'] = 'Patient143_T'
manual_dict['Patient148'] = 'Patient148_N'
manual_dict['Patient179'] = 'Patient179_T'
manual_dict['Patient181'] = 'Patient181_N'
manual_dict['Patient189'] = 'Patient189_T'
manual_dict['Patient31'] = 'Patient31_T_M'
manual_dict['Patient49'] = 'Patient49_N'
manual_dict['Patient82'] = 'Patient82_T_P'
manual_dict['Patient94'] = 'Patient94_N'


chosen_bio_list = []

for patient in patient_ids:
    
    if patient in manual_dict:
        bio2use = manual_dict[patient]
    else:
        bio_list = bio_dict[patient]
        bio_list.sort()
        bio2use = bio_list[1]
    
    chosen_bio_list += [bio2use]
    
    run_list_1 = run_dict[bio2use]
    
    hlas_1 = hla_str_dict[run_list_1]
    
    final_hla_dict[patient] = hlas_1

In [34]:
len(final_hla_dict)

144

In [35]:
df_s1 = pd.read_excel(data_dir+"/"+table_s1, 
                      sheet_name='Supplemental Table 1', 
                      skiprows=2, 
                      skipfooter=14)  
print(df_s1.shape)
df_s1[:6]

(144, 46)


,Unnamed: 0,total_muts,nonsyn_muts,clonal_muts,subclonal_muts,heterogeneity,total_neoantigens,CNA_prop,"gender (Male=1, Female=0)",biopsy site,...,postCTLA4,postMAPKTx,postCombinedCTLA_PD1,numPriorTherapies,biopsy site_categ,biopsyContext (1=Pre-Ipi; 2=On-Ipi; 3=Pre-PD1; 4=On-PD1),daysBiopsyToPD1,daysBiopsyAfterIpiStart,purity,ploidy
0,Patient1,34,22,12,10,0.454545,49,0.321417,0,skin,...,0.0,0.0,0,1,skin,3,-84,unk,0.92,1.73
1,Patient10,96,71,48,22,0.314286,230,0.391384,0,skin,...,1.0,1.0,0,2,skin,3,-12,107,0.83,1.84
2,Patient100,200,126,98,24,0.196721,301,0.029447,0,skin,...,0.0,0.0,0,1,skin,3,-94,33,0.11,2.17
3,Patient102,370,246,215,26,0.107884,825,0.169389,1,brain,...,0.0,0.0,0,0,brain,3,-64,na,0.70,3.24
4,Patient104,130,96,65,28,0.301075,329,0.206518,0,lymph node,...,0.0,0.0,0,0,lymph node,3,-57,na,0.86,4.58
5,Patient105,185,125,85,23,0.212963,334,0.394306,0,skin,...,0.0,0.0,1,0,skin,3,-22,na,0.86,2.42


In [36]:
set(df_s1['Unnamed: 0']) == set(df_ID['submitted_subject_id'])

True

In [37]:
set(df_s1['Unnamed: 0']) == set(final_hla_dict.keys())

True

In [38]:
hla_column = [final_hla_dict[x] for x in df_s1['Unnamed: 0'].tolist()]

In [39]:
df_s1["hla_class_ii_alleles"] = hla_column
df_s1

,Unnamed: 0,total_muts,nonsyn_muts,clonal_muts,subclonal_muts,heterogeneity,total_neoantigens,CNA_prop,"gender (Male=1, Female=0)",biopsy site,...,postMAPKTx,postCombinedCTLA_PD1,numPriorTherapies,biopsy site_categ,biopsyContext (1=Pre-Ipi; 2=On-Ipi; 3=Pre-PD1; 4=On-PD1),daysBiopsyToPD1,daysBiopsyAfterIpiStart,purity,ploidy,hla_class_ii_alleles
0,Patient1,34,22,12,10,0.454545,49,0.321417,0,skin,...,0.0,0,1,skin,3,-84,unk,0.92,1.73,"DRB1*04:01,DRB1*04:03,DQA1*03:01,DQA1*03:03,DQ..."
1,Patient10,96,71,48,22,0.314286,230,0.391384,0,skin,...,1.0,0,2,skin,3,-12,107,0.83,1.84,"DRB1*04:01,DRB1*15:01,DQA1*01:02,DQA1*03:01,DQ..."
2,Patient100,200,126,98,24,0.196721,301,0.029447,0,skin,...,0.0,0,1,skin,3,-94,33,0.11,2.17,"DRB1*01:01,DRB1*15:01,DQA1*01:01,DQA1*01:02,DQ..."
3,Patient102,370,246,215,26,0.107884,825,0.169389,1,brain,...,0.0,0,0,brain,3,-64,na,0.70,3.24,"DRB1*01:01,DRB1*11:04,DQA1*01:01,DQA1*05:05,DQ..."
4,Patient104,130,96,65,28,0.301075,329,0.206518,0,lymph node,...,0.0,0,0,lymph node,3,-57,na,0.86,4.58,"DRB1*01:01,DRB1*07:01,DQA1*01:02,DQA1*02:01,DQ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Patient9,911,652,562,80,0.124611,1996,0.111034,0,skin,...,0.0,0,0,skin,3,-39,na,0.88,2.02,"DRB1*04:01,DRB1*11:03,DQA1*03:03,DQA1*05:05,DQ..."
140,Patient94,414,268,227,21,0.084677,929,0.902095,0,skin,...,0.0,0,2,skin,3,-31,140,0.85,1.13,"DRB1*11:03,DRB1*15:01,DQA1*01:02,DQA1*05:05,DQ..."
141,Patient96,959,649,588,50,0.078370,2136,0.196499,1,pancreas,...,0.0,0,0,liver/visceral,3,-81,na,0.82,2.22,"DRB1*13:01,DRB1*15:01,DQA1*01:02,DQA1*01:03,DQ..."
142,Patient98,472,302,219,74,0.252560,790,0.223074,0,lung,...,0.0,1,0,lung,3,-24,na,0.60,1.99,"DRB1*15:01,DRB1*15:01,DQA1*01:02,DQA1*01:02,DQ..."


In [40]:
df_s1.isna().sum()

Unnamed: 0                                                  0
total_muts                                                  0
nonsyn_muts                                                 0
clonal_muts                                                 0
subclonal_muts                                              0
heterogeneity                                               0
total_neoantigens                                           0
CNA_prop                                                    0
gender (Male=1, Female=0)                                   0
biopsy site                                                 0
monthsBiopsyPreTx                                           0
BR                                                          0
PFS                                                         0
OS                                                          0
TimeToBR                                                    0
cyclesOnTherapy                                             0
txOngoin

In [41]:
df_s1.hla_class_ii_alleles.tolist()[0]

'DRB1*04:01,DRB1*04:03,DQA1*03:01,DQA1*03:03,DQB1*03:01,DQB1*03:02,DPA1*01:03,DPA1*01:03,DPB1*02:01,DPB1*14:01'

In [42]:
homozygous_flags = []

for hla_string in df_s1["hla_class_ii_alleles"].tolist():
    hlas = hla_string.split(",")
    if ((hlas[0]!= hlas[1]) and ((hlas[2]!=hlas[3]) or (hlas[4]!=hlas[5])) \
         and ((hlas[6]!= hlas[7]) or (hlas[8]!=hlas[9]))):
        homozygous_flags += [0]
    else:
        homozygous_flags += [1]

In [43]:
Counter(homozygous_flags)

Counter({0: 96, 1: 48})

In [44]:
48/144

0.3333333333333333

In [45]:
df_s1["homozygous"] = homozygous_flags

In [46]:
# only keep the subjects with hla ii alleles among those from emerson data
# or the hlas that we can find a replacement for from those from emerson data

df_pseudo = pd.read_csv("../data/for_encoders/HLA_II_pseudo_45.csv", header = 0)

In [47]:
df_pseudo.hla.tolist()

['HLA-DPAB*01:03_01:01',
 'HLA-DPAB*01:03_02:01',
 'HLA-DPAB*01:03_03:01',
 'HLA-DPAB*01:03_04:01',
 'HLA-DPAB*01:03_04:02',
 'HLA-DPAB*01:03_05:01',
 'HLA-DPAB*01:03_10:01',
 'HLA-DPAB*01:03_11:01',
 'HLA-DPAB*01:03_13:01',
 'HLA-DPAB*01:03_17:01',
 'HLA-DPAB*02:01_01:01',
 'HLA-DPAB*02:01_02:01',
 'HLA-DPAB*02:01_03:01',
 'HLA-DPAB*02:01_04:01',
 'HLA-DPAB*02:01_04:02',
 'HLA-DPAB*02:01_05:01',
 'HLA-DPAB*02:01_10:01',
 'HLA-DPAB*02:01_11:01',
 'HLA-DPAB*02:01_13:01',
 'HLA-DPAB*02:01_17:01',
 'HLA-DPAB*02:02_01:01',
 'HLA-DPAB*02:02_02:01',
 'HLA-DPAB*02:02_04:01',
 'HLA-DPAB*02:02_04:02',
 'HLA-DPAB*02:02_05:01',
 'HLA-DQAB*01:01_02:02',
 'HLA-DQAB*01:01_03:01',
 'HLA-DQAB*01:01_03:02',
 'HLA-DQAB*01:01_05:01',
 'HLA-DQAB*01:01_05:03',
 'HLA-DQAB*01:01_06:02',
 'HLA-DQAB*01:01_06:03',
 'HLA-DQAB*01:02_02:01',
 'HLA-DQAB*01:02_02:02',
 'HLA-DQAB*01:02_03:01',
 'HLA-DQAB*01:02_03:02',
 'HLA-DQAB*01:02_03:03',
 'HLA-DQAB*01:02_04:02',
 'HLA-DQAB*01:02_05:01',
 'HLA-DQAB*01:02_05:02',


In [48]:
'HLA-DPAB*01:03_01:01'[5]

'P'

In [47]:
DRB_set = set()
DQA_set = set()
DQB_set = set()
DPA_set = set()
DPB_set = set()

for cur_hla in df_pseudo.hla.tolist():
    if cur_hla[5] == 'R':
        DRB_set.add(cur_hla[4:])
    elif cur_hla[5] == 'Q':
        digit_list = cur_hla.split("*")[1].split("_")
        DQA_set.add("DQA1*"+digit_list[0])
        DQB_set.add("DQB1*"+digit_list[1])
    elif cur_hla[5] == 'P':
        digit_list = cur_hla.split("*")[1].split("_")
        DPA_set.add("DPA1*"+digit_list[0])
        DPB_set.add("DPB1*"+digit_list[1])
    else:
        print("unexpected character")

In [48]:
df_s1["hla_class_ii_alleles"].tolist()[:2]

['DRB1*04:01,DRB1*04:03,DQA1*03:01,DQA1*03:03,DQB1*03:01,DQB1*03:02,DPA1*01:03,DPA1*01:03,DPB1*02:01,DPB1*14:01',
 'DRB1*04:01,DRB1*15:01,DQA1*01:02,DQA1*03:01,DQB1*03:02,DQB1*06:02,DPA1*01:03,DPA1*01:03,DPB1*04:01,DPB1*04:01']

In [49]:
liu_2019_DRB_group_list = [x.split(",")[:2] for x in df_s1["hla_class_ii_alleles"].tolist()]
liu_2019_DQA_group_list = [x.split(",")[2:4] for x in df_s1["hla_class_ii_alleles"].tolist()]
liu_2019_DQB_group_list = [x.split(",")[4:6] for x in df_s1["hla_class_ii_alleles"].tolist()]
liu_2019_DPA_group_list = [x.split(",")[6:8] for x in df_s1["hla_class_ii_alleles"].tolist()]
liu_2019_DPB_group_list = [x.split(",")[8:] for x in df_s1["hla_class_ii_alleles"].tolist()]

liu_2019_DRB_list = [y for x in liu_2019_DRB_group_list for y in x]
liu_2019_DQA_list = [y for x in liu_2019_DQA_group_list for y in x]
liu_2019_DQB_list = [y for x in liu_2019_DQB_group_list for y in x]
liu_2019_DPA_list = [y for x in liu_2019_DPA_group_list for y in x]
liu_2019_DPB_list = [y for x in liu_2019_DPB_group_list for y in x]

print(len(liu_2019_DRB_list))
print(len(set(liu_2019_DRB_list)))

print(len(liu_2019_DQA_list))
print(len(set(liu_2019_DQA_list)))

print(len(liu_2019_DQB_list))
print(len(set(liu_2019_DQB_list)))

print(len(liu_2019_DPA_list))
print(len(set(liu_2019_DPA_list)))

print(len(liu_2019_DPB_list))
print(len(set(liu_2019_DPB_list)))

288
27
288
17
288
17
288
6
288
21


In [50]:
print(set(liu_2019_DRB_list) - DRB_set)
print(set(liu_2019_DQA_list) - DQA_set)
print(set(liu_2019_DQB_list) - DQB_set)
print(set(liu_2019_DPA_list) - DPA_set)
print(set(liu_2019_DPB_list) - DPB_set)

{'DRB1*08:77'}
{'DQA1*04:02', 'DQA1*05:48', 'DQA1*01:10', 'DQA1*02:14', 'DQA1*05:09'}
{'DQB1*06:280', 'DQB1*05:04', 'DQB1*03:457'}
{'DPA1*02:07', 'DPA1*02:06', 'DPA1*01:130'}
{'DPB1*1037:01', 'DPB1*02:02', 'DPB1*06:01', 'DPB1*14:01', 'DPB1*15:01', 'DPB1*51:01', 'DPB1*09:01', 'DPB1*104:01', 'DPB1*834:01', 'DPB1*784:01', 'DPB1*16:01', 'DPB1*19:01', 'DPB1*124:01'}


In [51]:
temp_DRB_candidate_list = list(set(liu_2019_DRB_list) - DRB_set)
temp_DRB_candidate_list.sort()
temp_DRB_candidate_list

['DRB1*08:77']

In [52]:
temp_DQA_candidate_list = list(set(liu_2019_DQA_list) - DQA_set)
temp_DQA_candidate_list.sort()
temp_DQA_candidate_list

['DQA1*01:10', 'DQA1*02:14', 'DQA1*04:02', 'DQA1*05:09', 'DQA1*05:48']

In [53]:
temp_DQB_candidate_list = list(set(liu_2019_DQB_list) - DQB_set)
temp_DQB_candidate_list.sort()
temp_DQB_candidate_list

['DQB1*03:457', 'DQB1*05:04', 'DQB1*06:280']

In [54]:
temp_DPA_candidate_list = list(set(liu_2019_DPA_list) - DPA_set)
temp_DPA_candidate_list.sort()
temp_DPA_candidate_list

['DPA1*01:130', 'DPA1*02:06', 'DPA1*02:07']

In [55]:
temp_DPB_candidate_list = list(set(liu_2019_DPB_list) - DPB_set)
temp_DPB_candidate_list.sort()
temp_DPB_candidate_list

['DPB1*02:02',
 'DPB1*06:01',
 'DPB1*09:01',
 'DPB1*1037:01',
 'DPB1*104:01',
 'DPB1*124:01',
 'DPB1*14:01',
 'DPB1*15:01',
 'DPB1*16:01',
 'DPB1*19:01',
 'DPB1*51:01',
 'DPB1*784:01',
 'DPB1*834:01']

In [56]:
DRB_replace_dict = defaultdict(str)
DQA_replace_dict = defaultdict(str)
DQB_replace_dict = defaultdict(str)
DPA_replace_dict = defaultdict(str)
DPB_replace_dict = defaultdict(str)

In [57]:
DRB_replace_dict['DRB1*08:77'] = 'DRB1*08:01'

In [58]:
DQA_replace_dict['DQA1*01:10'] = 'DQA1*01:01'
DQA_replace_dict['DQA1*02:14'] = 'DQA1*02:01'
DQA_replace_dict['DQA1*04:02'] = 'DQA1*04:01'
DQA_replace_dict['DQA1*05:09'] = 'DQA1*05:01'
DQA_replace_dict['DQA1*05:48'] = 'DQA1*05:01'

In [59]:
DQB_replace_dict['DQB1*03:457'] = 'DQB1*03:01'
DQB_replace_dict['DQB1*05:04'] = 'DQB1*05:01'
DQB_replace_dict['DQB1*06:280'] = 'DQB1*06:01'

In [60]:
DPA_replace_dict['DPA1*01:130'] = 'DPA1*01:03'
DPA_replace_dict['DPA1*02:06'] = 'DPA1*02:01'
DPA_replace_dict['DPA1*02:07'] = 'DPA1*02:01'

In [61]:
# no match is found for these 12 DPB1 alleles
#  'DPB1*06:01',
#  'DPB1*09:01',
#  'DPB1*1037:01',
#  'DPB1*104:01',
#  'DPB1*124:01',
#  'DPB1*14:01',
#  'DPB1*15:01',
#  'DPB1*16:01',
#  'DPB1*19:01',
#  'DPB1*51:01',
#  'DPB1*784:01',
# 'DPB1*834:01'

DPB_replace_dict['DPB1*02:02'] = 'DPB1*02:01'

In [62]:
# use a function to find matching hla allele from emerson data
# and build the dictionary again, to verify whether the previously built 
# dictionary is correct

# map each two digit HLA allele to its first match among the alleles


def create_dict(emerson_set, hlas2match):

    emerson_list = list(emerson_set)
    two_digit_dict = defaultdict(str)
    
    all_two_digits = list(set([x.split(":")[0] for x in emerson_list]))
    
    for cur_two in all_two_digits:
        
        cur_list = [x for x in emerson_list if x.split(":")[0]==cur_two]
        cur_list.sort()
        two_digit_dict[cur_two] = cur_list[0]
        
    result_dict = defaultdict(str)
    
    for temp_hla in hlas2match:
        if temp_hla.split(":")[0] in two_digit_dict:
            result_dict[temp_hla] = two_digit_dict[temp_hla.split(":")[0]]
    
    return result_dict


DRB_verify_dict = create_dict(DRB_set, temp_DRB_candidate_list)
DQA_verify_dict = create_dict(DQA_set, temp_DQA_candidate_list)
DQB_verify_dict = create_dict(DQB_set, temp_DQB_candidate_list)
DPA_verify_dict = create_dict(DPA_set, temp_DPA_candidate_list)
DPB_verify_dict = create_dict(DPB_set, temp_DPB_candidate_list)

In [63]:
print(len(DRB_replace_dict))
print(len(DQA_replace_dict))
print(len(DQB_replace_dict))
print(len(DPA_replace_dict))
print(len(DPB_replace_dict))

1
5
3
3
1


In [64]:
print(len(DRB_verify_dict))
print(len(DQA_verify_dict))
print(len(DQB_verify_dict))
print(len(DPA_verify_dict))
print(len(DPB_verify_dict))

1
5
3
3
1


In [65]:
# now we have verified that the translation dictionaries are correct

print(DRB_replace_dict == DRB_verify_dict)
print(DQA_replace_dict == DQA_verify_dict)
print(DQB_replace_dict == DQB_verify_dict)
print(DPA_replace_dict == DPA_verify_dict)
print(DPB_replace_dict == DPB_verify_dict)

True
True
True
True
True


In [ ]:
# get the list of hla alleles that either have pseudo sequences or can find a match

DRB_more = list(DRB_set) + list(DRB_replace_dict.keys())
DQA_more = list(DQA_set) + list(DQA_replace_dict.keys())
DQB_more = list(DQB_set) + list(DQB_replace_dict.keys())
DPA_more = list(DPA_set) + list(DPA_replace_dict.keys())
DPB_more = list(DPB_set) + list(DPB_replace_dict.keys())

hla_alleles_allowed = DRB_more+DQA_more+DQB_more+DPA_more+DPB_more
hla_alleles_allowed.sort()
print(len(hla_alleles_allowed))
print(len(set(hla_alleles_allowed)))

In [69]:
# we find match for certain alleles from liu 2019, but not for 12 of the DPB1 alleles

# count how many subjects from liu 2019 have hla alleles all under our consideration
# or belong to one of those that we can find match for

# for those excluded, print outthe DPB information

hla_alleles_allowed_set = set(hla_alleles_allowed)

subjects_to_consider = []

for name, hlas in zip(df_s1['Unnamed: 0'].tolist(), df_s1["hla_class_ii_alleles"].tolist()):
    cur_hla_set = set(hlas.split(","))
    if (len(cur_hla_set.intersection(hla_alleles_allowed_set)) == len(cur_hla_set)):
        subjects_to_consider += [name]
    else:
        print(name+": "+" ".join(hlas.split(",")[8:]))

Patient1: DPB1*02:01 DPB1*14:01
Patient104: DPB1*02:01 DPB1*09:01
Patient105: DPB1*02:01 DPB1*16:01
Patient108: DPB1*02:01 DPB1*06:01
Patient118: DPB1*04:01 DPB1*14:01
Patient143: DPB1*03:01 DPB1*1037:01
Patient150: DPB1*02:01 DPB1*834:01
Patient155: DPB1*06:01 DPB1*104:01
Patient156: DPB1*04:01 DPB1*19:01
Patient162: DPB1*04:01 DPB1*09:01
Patient187: DPB1*04:02 DPB1*104:01
Patient192: DPB1*04:02 DPB1*06:01
Patient196: DPB1*02:01 DPB1*09:01
Patient30: DPB1*04:01 DPB1*104:01
Patient35: DPB1*06:01 DPB1*104:01
Patient45: DPB1*04:01 DPB1*784:01
Patient46: DPB1*04:01 DPB1*104:01
Patient49: DPB1*04:02 DPB1*16:01
Patient58: DPB1*04:01 DPB1*51:01
Patient67: DPB1*04:01 DPB1*15:01
Patient7: DPB1*104:01 DPB1*104:01
Patient75: DPB1*04:01 DPB1*06:01
Patient80: DPB1*14:01 DPB1*14:01
Patient84: DPB1*02:01 DPB1*124:01


In [70]:
len(subjects_to_consider)

120

In [71]:
set(df_s1['Unnamed: 0'].tolist()) - set(subjects_to_consider)

{'Patient1',
 'Patient104',
 'Patient105',
 'Patient108',
 'Patient118',
 'Patient143',
 'Patient150',
 'Patient155',
 'Patient156',
 'Patient162',
 'Patient187',
 'Patient192',
 'Patient196',
 'Patient30',
 'Patient35',
 'Patient45',
 'Patient46',
 'Patient49',
 'Patient58',
 'Patient67',
 'Patient7',
 'Patient75',
 'Patient80',
 'Patient84'}

In [72]:
final_hla_dict['Patient1']

'DRB1*04:01,DRB1*04:03,DQA1*03:01,DQA1*03:03,DQB1*03:01,DQB1*03:02,DPA1*01:03,DPA1*01:03,DPB1*02:01,DPB1*14:01'

In [73]:
# subset patients to only keep those with HLAs from those from Emerson data
# or can find a match with pseudo information for
df_liu_2019_kept = df_s1.loc[df_s1['Unnamed: 0'].isin(subjects_to_consider)]
df_liu_2019_kept.shape

(120, 48)

In [74]:
Counter([a==b for a, b in zip(df_liu_2019_kept['Unnamed: 0'].tolist(), 
                              subjects_to_consider)])

Counter({True: 120})

In [75]:
df_liu_2019_kept[:2]

,Unnamed: 0,total_muts,nonsyn_muts,clonal_muts,subclonal_muts,heterogeneity,total_neoantigens,CNA_prop,"gender (Male=1, Female=0)",biopsy site,...,postCombinedCTLA_PD1,numPriorTherapies,biopsy site_categ,biopsyContext (1=Pre-Ipi; 2=On-Ipi; 3=Pre-PD1; 4=On-PD1),daysBiopsyToPD1,daysBiopsyAfterIpiStart,purity,ploidy,hla_class_ii_alleles,homozygous
1,Patient10,96,71,48,22,0.314286,230,0.391384,0,skin,...,0,2,skin,3,-12,107,0.83,1.84,"DRB1*04:01,DRB1*15:01,DQA1*01:02,DQA1*03:01,DQ...",1
2,Patient100,200,126,98,24,0.196721,301,0.029447,0,skin,...,0,1,skin,3,-94,33,0.11,2.17,"DRB1*01:01,DRB1*15:01,DQA1*01:01,DQA1*01:02,DQ...",1


In [76]:
df_liu_2019_kept.isna().sum()

Unnamed: 0                                                  0
total_muts                                                  0
nonsyn_muts                                                 0
clonal_muts                                                 0
subclonal_muts                                              0
heterogeneity                                               0
total_neoantigens                                           0
CNA_prop                                                    0
gender (Male=1, Female=0)                                   0
biopsy site                                                 0
monthsBiopsyPreTx                                           0
BR                                                          0
PFS                                                         0
OS                                                          0
TimeToBR                                                    0
cyclesOnTherapy                                             0
txOngoin

In [77]:
# translate hla alleles to those we have pseudo sequence for

kept_trans_alleles = []

def inner_translate(trans_dict, emerson_set, x):
    if x in emerson_set:
        return x
    elif x in trans_dict:
        return trans_dict[x]
    else:
        raise Exception(x+" is not in trans_dict")

for x in df_liu_2019_kept["hla_class_ii_alleles"].tolist():
    
    cur_alleles = x.split(",")
    
    DRB_trans = [inner_translate(DRB_replace_dict, DRB_set, cur_alleles[0]), 
                 inner_translate(DRB_replace_dict, DRB_set, cur_alleles[1])]
    DQA_trans = [inner_translate(DQA_replace_dict, DQA_set, cur_alleles[2]), 
                 inner_translate(DQA_replace_dict, DQA_set, cur_alleles[3])]
    DQB_trans = [inner_translate(DQB_replace_dict, DQB_set, cur_alleles[4]), 
                 inner_translate(DQB_replace_dict, DQB_set, cur_alleles[5])]  
    DPA_trans = [inner_translate(DPA_replace_dict, DPA_set, cur_alleles[6]), 
                 inner_translate(DPA_replace_dict, DPA_set, cur_alleles[7])]
    DPB_trans = [inner_translate(DPB_replace_dict, DPB_set, cur_alleles[8]), 
                 inner_translate(DPB_replace_dict, DPB_set, cur_alleles[9])] 
    
    cur_trans_alleles = DRB_trans + DQA_trans + DQB_trans + DPA_trans + DPB_trans
    kept_trans_alleles += [cur_trans_alleles]

In [78]:
len(set([y for x in kept_trans_alleles for y in x]))

63

In [79]:
len(set().union(*[DRB_set, DQA_set, DQB_set, DPA_set, DPB_set]))

78

In [80]:
len(set().union(*[DRB_set, DQA_set, DQB_set, DPA_set, DPB_set]) - set([y for x in kept_trans_alleles for y in x]))

15

In [81]:
kept_trans_alleles_string = [",".join(x) for x in kept_trans_alleles]
kept_trans_alleles_string[3:6]

['DRB1*03:01,DRB1*10:01,DQA1*01:05,DQA1*05:01,DQB1*02:01,DQB1*05:01,DPA1*01:03,DPA1*01:03,DPB1*02:01,DPB1*04:01',
 'DRB1*04:02,DRB1*07:01,DQA1*02:01,DQA1*03:01,DQB1*02:02,DQB1*03:02,DPA1*01:03,DPA1*01:03,DPB1*04:01,DPB1*04:02',
 'DRB1*01:01,DRB1*13:01,DQA1*01:01,DQA1*01:03,DQB1*05:01,DQB1*06:03,DPA1*01:03,DPA1*01:03,DPB1*02:01,DPB1*03:01']

In [82]:
df_liu_2019_kept["hla_class_ii_alleles"].tolist()[3:6]

['DRB1*03:01,DRB1*10:01,DQA1*01:05,DQA1*05:01,DQB1*02:01,DQB1*05:01,DPA1*01:03,DPA1*01:03,DPB1*02:01,DPB1*04:01',
 'DRB1*04:02,DRB1*07:01,DQA1*02:01,DQA1*03:01,DQB1*02:02,DQB1*03:02,DPA1*01:03,DPA1*01:03,DPB1*04:01,DPB1*04:02',
 'DRB1*01:01,DRB1*13:01,DQA1*01:01,DQA1*01:03,DQB1*05:01,DQB1*06:03,DPA1*01:03,DPA1*01:03,DPB1*02:01,DPB1*03:01']

In [83]:
df_liu_2019_kept['hla_class_ii_alleles_replace'] = kept_trans_alleles_string

/var/folders/2w/md4qmqxd3cn9kfl7flgbmyrw0000gp/T/ipykernel_26015/2729582048.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_liu_2019_kept['hla_class_ii_alleles_replace'] = kept_trans_alleles_string


In [84]:
list(zip(df_liu_2019_kept['hla_class_ii_alleles'], 
         df_liu_2019_kept['hla_class_ii_alleles_replace']))[:3]

[('DRB1*04:01,DRB1*15:01,DQA1*01:02,DQA1*03:01,DQB1*03:02,DQB1*06:02,DPA1*01:03,DPA1*01:03,DPB1*04:01,DPB1*04:01',
  'DRB1*04:01,DRB1*15:01,DQA1*01:02,DQA1*03:01,DQB1*03:02,DQB1*06:02,DPA1*01:03,DPA1*01:03,DPB1*04:01,DPB1*04:01'),
 ('DRB1*01:01,DRB1*15:01,DQA1*01:01,DQA1*01:02,DQB1*05:01,DQB1*06:02,DPA1*01:03,DPA1*01:03,DPB1*04:01,DPB1*04:01',
  'DRB1*01:01,DRB1*15:01,DQA1*01:01,DQA1*01:02,DQB1*05:01,DQB1*06:02,DPA1*01:03,DPA1*01:03,DPB1*04:01,DPB1*04:01'),
 ('DRB1*01:01,DRB1*11:04,DQA1*01:01,DQA1*05:05,DQB1*03:01,DQB1*05:01,DPA1*01:03,DPA1*01:03,DPB1*04:01,DPB1*04:02',
  'DRB1*01:01,DRB1*11:04,DQA1*01:01,DQA1*05:05,DQB1*03:01,DQB1*05:01,DPA1*01:03,DPA1*01:03,DPB1*04:01,DPB1*04:02')]

In [85]:
Counter([x==y for x,y in zip(df_liu_2019_kept['hla_class_ii_alleles_replace'].tolist(), 
                             kept_trans_alleles_string)])

Counter({True: 120})

In [86]:
df_liu_2019_kept.to_csv("../results/st18_liu_2019_kept_hla_ii_match_replace.csv", index = False)